In [1]:
import os
import numpy as np
import imageio
from morphing_pipeline import MorphingPipeline

In [2]:
os.environ['SPCONV_ALGO'] = 'native'

In [3]:
pipeline = MorphingPipeline(
    device="cuda",
)

Initializing barycenter optimizer...
Loading Trellis decoder...
[SPARSE] Backend: spconv, Attention: flash_attn
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading TRELLIS model: microsoft/TRELLIS-image-large...
[SPARSE][CONV] spconv algo: native
[ATTENTION] Using backend: flash_attn


/workspace/Wukong/.venv/lib/python3.11/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/workspace/Wukong/.venv/lib/python3.11/site-packages/spconv/pytorch/functional.py:96: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @_TORCH_CUSTOM_BWD
/workspace/Wukong/.venv/lib/python3.11/site-packages/spconv/pytorch/functional.py:162: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @_TORCH_CUSTOM_BWD
/workspace/Wukong/.venv/lib/python3.11/site-packages/spconv/pytorch/functional.py:242: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` i

TRELLIS pipeline loaded successfully!


## Poses5

In [5]:
# Пути к изображениям (замените на ваши пути)
source_image_path = "shape_000.jpg"
target_image_path = "shape_002.jpg"

In [14]:
results = pipeline.morph(
    source_image=source_image_path,
    target_image=target_image_path,
    num_steps=5,
)

Preprocessing source image...
Preprocessing target image...
Encoding source image...
Encoding target image...
Computing barycenter sequence (5 steps)...
Decoding latents to 3D...
  Decoding step 1/6...


Sampling: 100%|██████████| 25/25 [00:02<00:00,  8.51it/s]


  Decoding step 2/6...


Sampling: 100%|██████████| 25/25 [00:02<00:00,  8.56it/s]


  Decoding step 3/6...


...

  Decoding step 5/6...


Sampling: 100%|██████████| 25/25 [00:02<00:00,  8.54it/s]


  Decoding step 6/6...


Sampling: 100%|██████████| 25/25 [00:02<00:00,  8.49it/s]


Morphing completed!


In [7]:
output_dir = "output_poses5"
os.makedirs(output_dir, exist_ok=True)
print(f"\nSaving results to {output_dir}/...")
from trellis.utils import render_utils, postprocessing_utils

# Сохранение результатов
for i, outputs in enumerate(results):
    if outputs is None:
        print(f"  Step {i}: Skipping (no outputs)")
        continue
        
    print(f"  Step {i}: Saving results...")
    
    # Создаем GLB файл
    if 'mesh' in outputs and len(outputs['mesh']) > 0 and 'gaussian' in outputs and len(outputs['gaussian']) > 0:
        try:
            glb_path = os.path.join(output_dir, f"mesh_step_{i:03d}.glb")
            glb = postprocessing_utils.to_glb(
                outputs['gaussian'][0],
                outputs['mesh'][0],
                simplify=0.95,
                texture_size=1024,
            )
            glb.export(glb_path)
            print(f"    Saved GLB: {glb_path}")
        except Exception as e:
            print(f"    Warning: Could not save GLB: {e}")
    
    # Создаем видео из mesh
    if 'mesh' in outputs and len(outputs['mesh']) > 0:
        try:
            video_path = os.path.join(output_dir, f"mesh_step_{i:03d}.mp4")
            video = render_utils.render_video(outputs['mesh'][0], num_frames=120)['normal']
            imageio.mimsave(video_path, video, fps=30)
            print(f"    Saved video: {video_path}")
        except Exception as e:
            print(f"    Warning: Could not save video: {e}")
    
    # Создаем видео из gaussian если доступен
    if 'gaussian' in outputs and len(outputs['gaussian']) > 0:
        try:
            video_path = os.path.join(output_dir, f"gaussian_step_{i:03d}.mp4")
            video = render_utils.render_video(outputs['gaussian'][0], num_frames=120)['color']
            imageio.mimsave(video_path, video, fps=30)
            print(f"    Saved gaussian video: {video_path}")
        except Exception as e:
            print(f"    Warning: Could not save gaussian video: {e}")

print(f"\nAll results saved to {output_dir}/")


Saving results to output_poses5/...
  Step 0: Saving results...
Before postprocess: 90310 vertices, 180616 faces


Decimating Mesh: 100%|██████████[00:01<00:00]


After decimate: 4518 vertices, 9030 faces


Rasterizing: 100%|██████████| 1000/1000 [00:00<00:00, 1404.38it/s]


Found 3725 invisible faces
Dual graph: 13543 edges
Mincut solved, start checking the cut
Removed 4107 faces by mincut
INFO- Loaded 2471 vertices and 4923 faces.

100% done After remove invisible faces: 2473 vertices, 4946 faces


...

    Saved GLB: output_poses5/mesh_step_005.glb


Rendering: 120it [00:00, 133.64it/s]


    Saved video: output_poses5/mesh_step_005.mp4


Rendering: 120it [00:00, 279.35it/s]


    Saved gaussian video: output_poses5/gaussian_step_005.mp4

All results saved to output_poses5/


## Poses5 Intermediate

In [4]:
# Пути к изображениям (замените на ваши пути)
source_image_path = "shape_000.jpg"
target1_image_path = "shape_001.jpg"
target2_image_path = "shape_002.jpg"

In [5]:
results = pipeline.morph_three_points(
    source_image=source_image_path,
    target1_image=target1_image_path,
    target2_image=target2_image_path,
    num_steps=4,
)

Preprocessing images...
Encoding images...

=== Trajectory 1: Source -> Target1 (4 steps) ===
Decoding trajectory 1...
  Decoding step 1/5...


Sampling: 100%|██████████| 25/25 [00:03<00:00,  7.02it/s]


  Decoding step 2/5...


Sampling: 100%|██████████| 25/25 [00:03<00:00,  7.93it/s]


  Decoding step 3/5...


...

  Decoding step 4/5...


Sampling: 100%|██████████| 25/25 [00:02<00:00,  8.34it/s]


  Decoding step 5/5...


Sampling: 100%|██████████| 25/25 [00:02<00:00,  8.33it/s]



Three-point morphing completed!
  Trajectory 1 (Source->Target1): 5 frames
  Trajectory 2 (Source->Target2): 5 frames


In [6]:
output_dir = "output_poses5_intermediate"
os.makedirs(output_dir, exist_ok=True)
print(f"\nSaving results to {output_dir}/...")
from trellis.utils import render_utils, postprocessing_utils

# Сохранение результатов
for i, outputs in enumerate(results):
    if outputs is None:
        print(f"  Step {i}: Skipping (no outputs)")
        continue
        
    print(f"  Step {i}: Saving results...")
    
    # Создаем GLB файл
    if 'mesh' in outputs and len(outputs['mesh']) > 0 and 'gaussian' in outputs and len(outputs['gaussian']) > 0:
        try:
            glb_path = os.path.join(output_dir, f"mesh_step_{i:03d}.glb")
            glb = postprocessing_utils.to_glb(
                outputs['gaussian'][0],
                outputs['mesh'][0],
                simplify=0.95,
                texture_size=1024,
            )
            glb.export(glb_path)
            print(f"    Saved GLB: {glb_path}")
        except Exception as e:
            print(f"    Warning: Could not save GLB: {e}")
    
    # Создаем видео из mesh
    if 'mesh' in outputs and len(outputs['mesh']) > 0:
        try:
            video_path = os.path.join(output_dir, f"mesh_step_{i:03d}.mp4")
            video = render_utils.render_video(outputs['mesh'][0], num_frames=120)['normal']
            imageio.mimsave(video_path, video, fps=30)
            print(f"    Saved video: {video_path}")
        except Exception as e:
            print(f"    Warning: Could not save video: {e}")
    
    # Создаем видео из gaussian если доступен
    if 'gaussian' in outputs and len(outputs['gaussian']) > 0:
        try:
            video_path = os.path.join(output_dir, f"gaussian_step_{i:03d}.mp4")
            video = render_utils.render_video(outputs['gaussian'][0], num_frames=120)['color']
            imageio.mimsave(video_path, video, fps=30)
            print(f"    Saved gaussian video: {video_path}")
        except Exception as e:
            print(f"    Warning: Could not save gaussian video: {e}")

print(f"\nAll results saved to {output_dir}/")


Saving results to output_poses5_intermediate/...
  Step 0: Saving results...
Before postprocess: 100952 vertices, 201880 faces


Decimating Mesh: 100%|██████████[00:01<00:00]


After decimate: 5059 vertices, 10094 faces


Rasterizing: 100%|██████████| 1000/1000 [00:00<00:00, 1403.78it/s]


Found 4380 invisible faces
Dual graph: 15139 edges
Mincut solved, start checking the cut
Removed 5422 faces by mincut
INFO- Loaded 2365 vertices and 4672 faces.

100% done After remove invisible faces: 2517 vertices, 5034 faces


...

    Saved GLB: output_poses5_intermediate/mesh_step_009.glb


Rendering: 120it [00:00, 133.84it/s]


    Saved video: output_poses5_intermediate/mesh_step_009.mp4


Rendering: 120it [00:00, 294.23it/s]


    Saved gaussian video: output_poses5_intermediate/gaussian_step_009.mp4

All results saved to output_poses5_intermediate/
